<a href="https://colab.research.google.com/github/ailiskab-hub/Prot_BERT_for_prediction_protein_localization/blob/main/Pred_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
import datetime
import random
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [5]:
import tensorflow as tf
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )

In [6]:
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Prot_bert_clf/")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30, 1024, padding_idx=0)
      (position_embeddings): Embedding(40000, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-29): 30 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,)

In [7]:
def add_spases(s):
        # Добавляем пробелы между АК (нужно для токенизации)
  s_ = ''
  for i in s:
    s_ += i+' '
  return s_

In [8]:
sequence = 'MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDLVNATDRELVKFIGDIYYGLFRGCKVRQCGLGGRQSQFTIFPHLVKELNAGLHVMILLLLFLALALALVSMGFAILNMIQVPYRAVSGPGGICLWNVLAGGVVALAIASFVAAVKFHDLTERIANFQEKLFQFVVVEEQYEESFWICVASASAHAANLVVVAISQIPLPEIKTKIEEATVTAEDILY' # @param {type:"raw"}

In [9]:
sequence = re.sub(r"[UZOB]", "X", sequence)
sequence = add_spases(sequence)

In [10]:
sequence

'M P G W F K K A W Y G L A S L L S F S S F I L I I V A L V V P H W L S G K I L C Q T G V D L V N A T D R E L V K F I G D I Y Y G L F R G C K V R Q C G L G G R Q S Q F T I F P H L V K E L N A G L H V M I L L L L F L A L A L A L V S M G F A I L N M I Q V P Y R A V S G P G G I C L W N V L A G G V V A L A I A S F V A A V K F H D L T E R I A N F Q E K L F Q F V V V E E Q Y E E S F W I C V A S A S A H A A N L V V V A I S Q I P L P E I K T K I E E A T V T A E D I L Y '

In [11]:
print(' Original: ', sequence)

print('Tokenized: ', tokenizer.tokenize(sequence))

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sequence)))

 Original:  M P G W F K K A W Y G L A S L L S F S S F I L I I V A L V V P H W L S G K I L C Q T G V D L V N A T D R E L V K F I G D I Y Y G L F R G C K V R Q C G L G G R Q S Q F T I F P H L V K E L N A G L H V M I L L L L F L A L A L A L V S M G F A I L N M I Q V P Y R A V S G P G G I C L W N V L A G G V V A L A I A S F V A A V K F H D L T E R I A N F Q E K L F Q F V V V E E Q Y E E S F W I C V A S A S A H A A N L V V V A I S Q I P L P E I K T K I E E A T V T A E D I L Y 
Tokenized:  ['M', 'P', 'G', 'W', 'F', 'K', 'K', 'A', 'W', 'Y', 'G', 'L', 'A', 'S', 'L', 'L', 'S', 'F', 'S', 'S', 'F', 'I', 'L', 'I', 'I', 'V', 'A', 'L', 'V', 'V', 'P', 'H', 'W', 'L', 'S', 'G', 'K', 'I', 'L', 'C', 'Q', 'T', 'G', 'V', 'D', 'L', 'V', 'N', 'A', 'T', 'D', 'R', 'E', 'L', 'V', 'K', 'F', 'I', 'G', 'D', 'I', 'Y', 'Y', 'G', 'L', 'F', 'R', 'G', 'C', 'K', 'V', 'R', 'Q', 'C', 'G', 'L', 'G', 'G', 'R', 'Q', 'S', 'Q', 'F', 'T', 'I', 'F', 'P', 'H', 'L', 'V', 'K', 'E', 'L', 'N', 'A', 'G', 'L', 'H', 'V', 'M', 'I', 'L', 

In [41]:
def predict_lcz(sequence, printable=True):
  encoded_dict = tokenizer.encode_plus(sequence, add_special_tokens = True,
                          max_length = 350, padding=True,
                          return_attention_mask = True, return_tensors = 'pt')

  input_ids = encoded_dict['input_ids']
  attention_masks = encoded_dict['attention_mask']

  model.eval()

  input_ids = input_ids.to(device)
  attention_masks= attention_masks.to(device)

  with torch.no_grad():
    outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  res = np.argmax(logits).flatten()[0]

  if printable:
    print('DONE.')
    print(f'Resuls: class {res}')
  else:
    return res

In [42]:
predict_lcz(sequence)

DONE.
Resuls: class 2
